In [1]:
import numpy as np
import pandas as pd
import os
import scipy.io

In [2]:
main_path=('../data/')

In [3]:
meta_pd=pd.read_csv(os.path.join(main_path,'metadata.csv'))

In [4]:
meta_pd

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,charge,2008-04-02 13:08:17,24,B0005,0,1,00001.csv,NaN,NaN,NaN
1,discharge,2008-04-02 15:25:41,24,B0005,1,2,00002.csv,1.856487,NaN,NaN
2,charge,2008-04-02 16:37:51,24,B0005,2,3,00003.csv,NaN,NaN,NaN
3,discharge,2008-04-02 19:43:48,24,B0005,3,4,00004.csv,1.846327,NaN,NaN
4,charge,2008-04-02 20:55:40,24,B0005,4,5,00005.csv,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1546,impedance,2008-08-20 01:34:44,24,B0018,314,1547,01547.csv,NaN,0.066069,0.088959
1547,charge,2008-08-20 02:06:37,24,B0018,315,1548,01548.csv,NaN,NaN,NaN
1548,discharge,2008-08-20 05:02:00,24,B0018,316,1549,01549.csv,1.354797,NaN,NaN
1549,charge,2008-08-20 05:49:31,24,B0018,317,1550,01550.csv,NaN,NaN,NaN


In [5]:
meta_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551 entries, 0 to 1550
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   type                 1551 non-null   object 
 1   start_time           1551 non-null   object 
 2   ambient_temperature  1551 non-null   int64  
 3   battery_id           1551 non-null   object 
 4   test_id              1551 non-null   int64  
 5   uid                  1551 non-null   int64  
 6   filename             1551 non-null   object 
 7   Capacity             468 non-null    float64
 8   Re                   609 non-null    float64
 9   Rct                  609 non-null    float64
dtypes: float64(3), int64(3), object(4)
memory usage: 121.3+ KB


In [6]:
meta_pd['battery_id'].unique()

array(['B0005', 'B0006', 'B0018'], dtype=object)

In [7]:
data1_pd=pd.read_csv(os.path.join(main_path,'cleaned_dataset','00003.csv'))
data2_pd=pd.read_csv(os.path.join(main_path,'cleaned_dataset','00005.csv'))

In [8]:
data1_pd.head()

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time
0,3.325055,0.000302,29.341851,-0.002,0.003,0.000
1,3.001951,-3.361983,29.335723,-3.370,1.414,2.516
2,3.434644,1.508670,29.334717,1.498,4.142,5.500
3,3.454857,1.510043,29.341949,1.498,4.180,8.391
4,3.468788,1.508704,29.331462,1.498,4.196,11.266


In [9]:
data2_pd.head()

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time
0,3.352604,0.001990,29.553301,0.000,0.003,0.000
1,3.035879,-3.384408,29.538751,-3.387,1.419,2.547
2,3.459468,1.510318,29.539000,1.498,4.175,5.547
3,3.478333,1.511211,29.543794,1.498,4.213,8.391
4,3.491893,1.513270,29.543092,1.498,4.229,11.281


In [10]:
meta_pd['Capacity'] = pd.to_numeric(meta_pd['Capacity'], errors='coerce')

In [11]:
filtered_df=meta_pd[meta_pd['type']!='impedance']
filtered_df=filtered_df.fillna(1e-6)

In [12]:
c=0
for idx,row in meta_pd[meta_pd['type']=='charge'].iterrows():
    file = pd.read_csv(os.path.join(main_path,'cleaned_dataset',row['filename']))
    c=max(c,max(file['Current_measured']))
print(c)

1.536047031556956


In [13]:
capacities_df=meta_pd[meta_pd['type']=='discharge'][['battery_id','Capacity']]

In [14]:
capacities_df

,battery_id,Capacity
1,B0005,1.856487
3,B0005,1.846327
5,B0005,1.835349
7,B0005,1.835263
9,B0005,1.834646
...,...,...
1540,B0018,1.362737
1542,B0018,1.363405
1545,B0018,1.351865
1548,B0018,1.354797


In [15]:
cap={}
for bid in capacities_df['battery_id'].unique():
    battery_df = capacities_df[(capacities_df['battery_id'] == bid)]
    battery_df['Capacity'] = pd.to_numeric(battery_df['Capacity'], errors='coerce')
    if 'Capacity' in battery_df.columns:
        capacity = battery_df['Capacity'].dropna().max()
    else:
        capacity = 0.0
    cap[bid] = capacity

C:\Users\kurt_\AppData\Local\Temp\ipykernel_5592\3023381454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  battery_df['Capacity'] = pd.to_numeric(battery_df['Capacity'], errors='coerce')


In [16]:
dfs = []
for idx, row in filtered_df.iterrows():
    file_path = os.path.join(main_path,'cleaned_dataset',row['filename'])
    df = pd.read_csv(file_path)
    df=df[['Voltage_measured','Current_measured','Temperature_measured','Time']]
    df['battery_id']=row['battery_id']
    df['is_charging']= (row['type']=='charge')*1
    if row['type']=='discharge':
        df['Capacity']=row['Capacity']
    else:
        df['Capacity']=cap[row['battery_id']]
    dfs.append(df)
combined_df = pd.concat(dfs, ignore_index=True)

print(combined_df.shape)

(1497592, 7)


In [17]:
combined_df.isnull().sum()

Voltage_measured        2
Current_measured        2
Temperature_measured    2
Time                    0
battery_id              0
is_charging             0
Capacity                0
dtype: int64

In [18]:
combined_df=combined_df.dropna()

In [19]:
combined_df

,Voltage_measured,Current_measured,Temperature_measured,Time,battery_id,is_charging,Capacity
0,3.873017,-0.001201,24.655358,0.000,B0005,1,1.856487
1,3.479394,-4.030268,24.666480,2.532,B0005,1,1.856487
2,4.000588,1.512731,24.675394,5.500,B0005,1,1.856487
3,4.012395,1.509063,24.693865,8.344,B0005,1,1.856487
4,4.019708,1.511318,24.705069,11.125,B0005,1,1.856487
...,...,...,...,...,...,...,...
1497587,3.443760,-0.002426,35.383979,2686.359,B0018,0,1.341051
1497588,3.453271,-0.000981,35.179732,2700.546,B0018,0,1.341051
1497589,3.461963,0.000209,34.977000,2714.640,B0018,0,1.341051
1497590,3.469907,0.001516,34.785943,2728.750,B0018,0,1.341051


In [20]:
combined_df.to_csv('../data/combined_df.csv', index=False)

In [21]:
combined_df=pd.read_csv('../data/combined_df.csv')

In [22]:
def compute_soc_and_delta(df, time_col='Time', current_col='Current_measured',
                          flag_col='is_charging', battery_col='battery_id',
                          initial_soc=0.5, cap_col='Capacity'):

    df = df.copy()
    
    # delta_t
    df = df.sort_values([battery_col, flag_col, time_col]).reset_index(drop=True)
    df['delta_t'] = df.groupby([battery_col, flag_col])[time_col].diff()
    df['delta_t'] = df['delta_t'].apply(lambda x: x if pd.notna(x) and x > 0 else 1e-4)

    # delta_Ah
    df['delta_Ah'] = (df[current_col] * df['delta_t']) / 3600.0
    df['delta_Ah_signed'] = df['delta_Ah'] * df[flag_col].apply(lambda x: -1 if x else 1)

    # Cumulative Ah
    df['comb_Ah'] = df.groupby(battery_col)['delta_Ah_signed'].cumsum()

    battery_capacity = df.groupby(battery_col)[cap_col].max()
    battery_capacity = battery_capacity[battery_capacity > 0].dropna()
    battery_capacity = battery_capacity.to_dict()
    df['battery_capacity'] = df[battery_col].map(battery_capacity)

    if df['battery_capacity'].isna().any():
        raise ValueError("Missing Battery Capacity.")

    # SoC
    df['SoC'] = initial_soc + (df['comb_Ah'] / df['battery_capacity'])
    df['SoC'] = df['SoC'].clip(0, 1)

    df.drop(['delta_Ah', 'delta_Ah_signed', 'comb_Ah', 'battery_capacity'], axis=1, inplace=True)
    return df


In [23]:
combined_df = compute_soc_and_delta(
    combined_df,
    time_col='Time',
    current_col='Current_measured',
    flag_col='is_charging',
    battery_col='battery_id',
    initial_soc=0.5,
    cap_col='Capacity'
)

print(combined_df[['battery_id', 'Time', 'SoC', 'delta_t']].head(10))
print("Max SoC:", combined_df['SoC'].max())
print("Min SoC:", combined_df['SoC'].min())

  battery_id  Time  SoC  delta_t
0      B0005   0.0  0.5   0.0001
1      B0005   0.0  0.5   0.0001
2      B0005   0.0  0.5   0.0001
3      B0005   0.0  0.5   0.0001
4      B0005   0.0  0.5   0.0001
5      B0005   0.0  0.5   0.0001
6      B0005   0.0  0.5   0.0001
7      B0005   0.0  0.5   0.0001
8      B0005   0.0  0.5   0.0001
9      B0005   0.0  0.5   0.0001
Max SoC: 0.5000019442423265
Min SoC: 0.0


In [24]:
print(combined_df['SoC'].max())
print(combined_df['SoC'].min())

0.5000019442423265
0.0


In [25]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497590 entries, 0 to 1497589
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Voltage_measured      1497590 non-null  float64
 1   Current_measured      1497590 non-null  float64
 2   Temperature_measured  1497590 non-null  float64
 3   Time                  1497590 non-null  float64
 4   battery_id            1497590 non-null  object 
 5   is_charging           1497590 non-null  int64  
 6   Capacity              1497590 non-null  float64
 7   delta_t               1497590 non-null  float64
 8   SoC                   1497590 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 102.8+ MB


In [26]:
batteries = combined_df['battery_id'].unique()
np.random.seed(42)
np.random.shuffle(batteries)

# 70% train / 30% test
n = len(batteries)
train_ids = batteries[: int(0.7 * n)]
test_ids  = batteries[int(0.7 * n):]

train_df = combined_df[combined_df['battery_id'].isin(train_ids)]
test_df  = combined_df[combined_df['battery_id'].isin(test_ids)]

print(f"Train batteries: {train_ids}")
print(f"Test batteries: {test_ids}")

Train batteries: ['B0005' 'B0006']
Test batteries: ['B0018']


In [27]:
save_path = '../data/model_data/'
os.makedirs(save_path, exist_ok=True)
train_df.to_csv(os.path.join(save_path, "train_df.csv"), index=False)
test_df.to_csv(os.path.join(save_path, "test_df.csv"), index=False)